In [1]:
from preamble import *
from scipy.stats import pearsonr
from sklearn.preprocessing import normalize

In [2]:
pdo = pd.read_csv('Data/PDO/data_PDO.csv')

In [3]:
duration = pd.read_csv('Data/computed-data/computed_annual-intensification-duration_JTWC.csv')

### Monthly Correlation

In [4]:
pdo_correlation = []

for i in (np.arange(12)+1):
    # Extract pdo temperature for exactly one month each year
    temp = pdo[pdo['month']==i]['PDO'].values
    # Calculate correlation with duration
    pdo_correlation.append(pearsonr(duration['Intensification Duration'],temp)[0])

In [5]:
corr_table = pd.DataFrame({'month':(np.arange(12)+1),'corr_pdo':pdo_correlation})

In [6]:
monthly_corr = corr_table.set_index('month')
monthly_corr.round(2).transpose()

month,1,2,3,4,5,6,7,8,9,10,11,12
corr_pdo,0.32,0.2,0.24,0.31,0.23,0.2,0.2,0.3,0.29,0.27,0.31,0.31


### Quarterly Correlation

In [7]:
q_pdo_correlation = []

for i in (np.arange(4)):
    # Extract pdo temperature for each quarter each year
    temp = pdo[pdo['month'].between(3*i+1,3*(i+1))].groupby('YEAR').mean()['PDO'].values
    # Calculate correlation with duration
    q_pdo_correlation.append(pearsonr(duration['Intensification Duration'],temp)[0])

In [8]:
q_corr_table = pd.DataFrame({'quarter':['Jan-Mar','Apr-Jun','Jul-Sep','Oct-Dec'],'corr_pdo':q_pdo_correlation})

In [9]:
q_corr = q_corr_table.set_index('quarter')
q_corr.round(2).transpose()

quarter,Jan-Mar,Apr-Jun,Jul-Sep,Oct-Dec
corr_pdo,0.27,0.26,0.29,0.32


### Annual Correlation

In [10]:
y_pdo_correlation = []

# Calculate correlation of annual pdo temperature with duration
print('Annual PDO correlation coefficient: {:.3f}'.format(pearsonr(duration['Intensification Duration'],
                                                pdo.groupby('YEAR').mean()['PDO'].values)[0]))

Annual PDO correlation coefficient: 0.350


### July-Oct Correlation

In [11]:
# Calculate correlation with duration
print ('Nino4 correlation July-Oct: {:.3f}'.format((pearsonr(duration['Intensification Duration'],pdo[pdo['month'].between(7,10)].groupby('YEAR').mean()['PDO'].values)[0])))

Nino4 correlation July-Oct: 0.295


In [12]:
# Extract the month of occurrence of cyclones
s_duration = pd.read_csv('Data/computed-data/computed_cyclone-metrics_JTWC.csv')
month_data = pd.read_csv('Data/IBTrACS/data_ibtracs_JTWC.csv')
def find_month(data):
    return data.loc[data['VMAX'].idxmax()]['S_TIME']

month_data = month_data.groupby('CycNo').apply(find_month)
month_data = pd.to_datetime('1858-11-17', format='%Y-%m-%d') + pd.to_timedelta(month_data, unit='d')
month_data = month_data.to_frame().reset_index()
month_data.columns = ['CycNo','TIME']
month_data['MONTH'] = month_data['TIME'].apply(lambda x: x.month)

In [13]:
s_duration = s_duration.merge(month_data, on='CycNo', how='inner')

In [14]:
b = np.array([
              [7,12,7,12],
              [7,9,1,12],
              [8,12,1,12],
            [8,11,1,12],
            ])

In [15]:
for item in b:
    season7_10 = pdo[pdo['month'].between(item[0],item[1])].groupby('YEAR').mean()['PDO'].values
    duration7_10 = s_duration[s_duration['MONTH'].between(item[2],item[3])].groupby('YEAR')['Intensification Duration'].mean().values

    try:
        a = pearsonr(duration7_10, season7_10)[0]
        if a > 0:
            print(item)
            print(a)
    except:
        pass

[ 7 12  7 12]
0.263900262054
[ 7  9  1 12]
0.286296034865
[ 8 12  1 12]
0.338169435803
[ 8 11  1 12]
0.327108669356
